# 0 Import Packages

In [ ]:
import os
import csv
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
import pandas as pd
#!pip install --upgrade --force-reinstall --no-deps kaggle

# 1 Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2 Connect to Kaggle and Download Data

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir /root/.kaggle

import json
token = {"username":"yanyuc","key":"dc3249c0209ecf021c2a7c30ff21d247"} # This is personal user name and token
!pwd
with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(token, file)
for i in range(2):
  !chmod 600 /root/.kaggle/kaggle.json
  !cp /root/.kaggle/kaggle.json -/.kaggle/
  !kaggle config set -n path -v /root
  !cat /root/.kaggle/kaggle.json


In [ ]:
!kaggle competitions download --force -c 11-785-s22-hw1p2
!unzip /root/competitions/11-785-s22-hw1p2/11-785-s22-hw1p2.zip

# 3 Define Components

## Network

In [ ]:
from torch.nn.modules.batchnorm import BatchNorm1d
from torch.nn.modules.dropout import Dropout
from torch.nn.modules.activation import ReLU
from torch.nn.modules.activation import Mish

class Network(torch.nn.Module):
    def __init__(self, context):
        super(Network, self).__init__()
        # TODO: Please try different architectures
        in_size = (2 * context + 1) * 13
        layers = [
            nn.Linear(in_size, 4096),
            nn.BatchNorm1d(4096),
            nn.Mish(),
            nn.Dropout(0.3),

            nn.Linear(4096,4096),
            nn.BatchNorm1d(4096),
            nn.Mish(),
            nn.Dropout(0.3),

            nn.Linear(4096, 4096),
            nn.BatchNorm1d(4096),
            nn.Mish(),
            nn.Dropout(0.2),

            nn.Linear(4096,4096),
            nn.BatchNorm1d(4096),
            nn.Mish(),
            nn.Dropout(0.2),

            nn.Linear(4096,4096),
            nn.BatchNorm1d(4096),
            nn.Mish(),
            nn.Dropout(0.1),

            nn.Linear(4096,4096),
            nn.BatchNorm1d(4096),
            nn.Mish(),
            nn.Dropout(0.1),

            nn.Linear(4096,40)
        ]

        for i in range(0, len(layers), 4):
          nn.init.kaiming_normal_(layers[i].weight, mode='fan_in') 

        self.layers = nn.Sequential(*layers)

    def forward(self, A0):
        x = self.layers(A0)
        return x

## LibriSample and LibriItems

In [ ]:

class LibriSamples(torch.utils.data.Dataset):
    def __init__(self, data_path, sample=28539, shuffle=True, partition="dev-clean", csvpath=None, train=True): # The original sample in the starter notebook was 20000
        # sample represent how many npy files will be preloaded for one __getitem__ call
        self.sample = sample 
        
        self.X_dir = data_path + "/" + partition + "/mfcc/"
        self.Y_dir = data_path + "/" + partition +"/transcript/"
        
        self.X_names = os.listdir(self.X_dir)
        self.Y_names = os.listdir(self.Y_dir)

        # using a small part of the dataset to debug
        if csvpath:
          if train:
            subset = self.parse_csv(csvpath)
            self.X_names = [i for i in self.X_names if i in subset]
            self.Y_names = [i for i in self.Y_names if i in subset]
          else:
            self.X_names = list(pd.read_csv(csvpath).file)

        
        if shuffle == True:
            XY_names = list(zip(self.X_names, self.Y_names))
            random.shuffle(XY_names)
            self.X_names, self.Y_names = zip(*XY_names)
        
        assert(len(self.X_names) == len(self.Y_names))
        self.length = len(self.X_names)
        
        self.PHONEMES = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']
      
    @staticmethod
    def parse_csv(filepath):
        subset = []
        with open(filepath) as f:
            f_csv = csv.reader(f)
            for row in f_csv:
                subset.append(row[1])
        return subset[1:]

    def __len__(self):
        return int(np.ceil(self.length / self.sample))
        
    def __getitem__(self, i):
        sample_range = range(i*self.sample, min((i+1)*self.sample, self.length))
        
        X, Y = [], []
        for j in sample_range:
            X_path = self.X_dir + self.X_names[j]
            Y_path = self.Y_dir + self.Y_names[j]
            
            label = [self.PHONEMES.index(yy) for yy in np.load(Y_path)][1:-1]

            X_data = np.load(X_path)
            X_data = (X_data - X_data.mean(axis=0))/X_data.std(axis=0)
            X.append(X_data)
            Y.append(np.array(label))
            
        X, Y = np.concatenate(X), np.concatenate(Y)
        return X, Y
    
class LibriItems(torch.utils.data.Dataset):
    def __init__(self, X, Y, context = 0):
        assert(X.shape[0] == Y.shape[0])
        
        self.length  = X.shape[0]
        self.context = context

        if context == 0:
            self.X, self.Y = X, Y
        else:
            X = np.pad(X, ((context,context), (0,0)), 'constant', constant_values=(0,0))
            self.X, self.Y = X, Y
        
    def __len__(self):
        return self.length
        
    def __getitem__(self, i):
        if self.context == 0:
            xx = self.X[i].flatten()
            yy = self.Y[i]
        else:
            xx = self.X[i:(i + 2*self.context + 1)].flatten()
            yy = self.Y[i]
        return xx, yy
    



## Train and Test

In [ ]:


def train(args, model, device, train_samples, optimizer, criterion, epoch):
    model.train()
    for i in range(len(train_samples)):
        X, Y = train_samples[i]
        train_items = LibriItems(X, Y, context=args['context'])
        train_loader = torch.utils.data.DataLoader(train_items, batch_size=args['batch_size'], shuffle=True)

        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.float().to(device)
            target = target.long().to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % args['log_interval'] == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))


def test(args, model, device, dev_samples):
    model.eval()
    true_y_list = []
    pred_y_list = []
    with torch.no_grad():
        for i in range(len(dev_samples)):
            X, Y = dev_samples[i]

            test_items = LibriItems(X, Y, context=args['context'])
            test_loader = torch.utils.data.DataLoader(test_items, batch_size=args['batch_size'], shuffle=False)

            for data, true_y in test_loader:
                data = data.float().to(device)
                true_y = true_y.long().to(device)                
                
                output = model(data)
                pred_y = torch.argmax(output, axis=1)

                pred_y_list.extend(pred_y.tolist())
                true_y_list.extend(true_y.tolist())

    train_accuracy =  accuracy_score(true_y_list, pred_y_list)
    return train_accuracy


def main(args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model = Network(args['context']).to(device)
    model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/best_model_hw1p2_6')) # Read the saved model
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=args['lr'])
    criterion = torch.nn.CrossEntropyLoss()
    # If you want to use full Dataset, please pass None to csvpath
    train_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="train-clean-100", csvpath=None)
    dev_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="dev-clean")

    for epoch in range(1, args['epoch'] + 1):
        train(args, model, device, train_samples, optimizer, criterion, epoch)
        test_acc = test(args, model, device, dev_samples)
        print('Dev accuracy ', test_acc)
        
        # Compare to decide if to save the model
        if test_acc > 0.865: # The best accuracy is hard coded here because there were only few changes
          # save the model to specified directory
          torch.save(model.state_dict(),'/content/drive/MyDrive/Colab Notebooks/best_model_hw1p2_6') 


if __name__ == '__main__':
    args = {
        'batch_size': 4096,
        'context': 30,
        'log_interval': 200,
        'LIBRI_PATH': '/content/hw1p2_student_data',
        'lr': 0.0009,
        'epoch': 25
    }
    main(args)

# 4 Submission Validation

In [ ]:
args = {
        'batch_size': 4096,
        'context': 30,
        'log_interval': 200,
        'LIBRI_PATH': '/content/hw1p2_student_data',
        'lr': 0.0009,
        'epoch': 25
    }
context = args['context']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Network(args['context'])
# Load the best model. If you want to use the model just trained, make next line a comment
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/best_model_hw1p2_6')) 
model.to(device)

## Load the test data

In [ ]:
# Referenced from LibriItems class
class ValidationItems(torch.utils.data.Dataset):
    def __init__(self, X, context = 0):
        
        self.length  = X.shape[0]
        self.context = context

        if context == 0:
            self.X = X
        else:
            self.X = np.pad(X, ((self.context,self.context), (0,0)), 'constant', constant_values=(0, 0)) 
        
    def __len__(self):
        return self.length
        
    def __getitem__(self, i):
        if self.context == 0:
            xx = self.X[i].flatten()
        else:
            xx = self.X[i:(i + 2 * self.context + 1)].flatten()
        return xx



In [ ]:
# Referenced from LibriSample class and Test() method
def validate(args, model, device, lib_path):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.eval()
  X_list = []
  Y_list = []

  X_names = os.listdir(lib_path)
  X_names.sort()

  for i in range(len(X_names)):
    X_path = lib_path + X_names[i]
    X_data = np.load(X_path)
    X_data = (X_data - X_data.mean(axis=0))/X_data.std(axis=0)
    X_list.append(X_data)
  
  X_list = np.concatenate(X_list)


  with torch.no_grad():
      validation_items = ValidationItems(X_list, context=args['context'])
      validation_loader = torch.utils.data.DataLoader(validation_items, batch_size = args['batch_size'], shuffle=False)
      
      for data in validation_loader:
        data = data.float().to(device)
        output = model(data)
        Y = torch.argmax(output, axis = 1)
        Y_list += Y.tolist()
      

  return Y_list


In [ ]:
lib_path = "/content/hw1p2_student_data/test-clean/mfcc/" # Argument for prediction
prediction = validate(args, model, device, lib_path) # Predict the Test data


## Submit to kaggle

In [ ]:
submission_data = pd.DataFrame(data = {'Id': [i for i in range(len(prediction))], 'Label':prediction}, index=None) # Zip the prediction label and id into one dataframe
submission_data.to_csv("submission.csv",index=0) 
! kaggle competitions submit -c 11-785-s22-hw1p2 -f submission.csv -m "yanyuc"